<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=fe007b9d84ce3824a1bcc69a4dc4ab5f42d2f803ffbaf3ea594a94a75d68c96d
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-23 10:41:33
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.43 C
-------------------
Today PnL: 93.15 K (0.66%)
Current PnL: -19.15 L (-12.76%)
CY Booked + Current PnL: -7.05 L (-4.7%)
-------------------
Total profit:  2.46 L
Total loss:  -21.62 L
-------------------
Total Booked + Current PnL: 19.93 L (16.18%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.49%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 86.35 L (60.59%)
Deployed:  1.23 C
Current:  1.43 C
CAGR/XIRR %: 8.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
58    RELIANCE   1533.0    -8.78    73.0       X-LC     3.06  228727.0   
85      VOLTAS   1530.0     2.98    61.0       X-MC     2.43  216195.0   
2   ABBOTINDIA  35195.0    -8.43    48.0       X-MC     3.47   90540.0   
42         ITC    452.0   -36.23    67.0       X-LC     2.09  205380.0   
78  TTKPRESTIG    770.0    96.58    50.0       M-SC     1.90   84659.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
58      18361.0  10430.0        0.07          8.73   4.56  13.68     37.0   
85      24453.0  13296.0       -1.00         12.75   6.15  19.69     99.0   
2        -138.0  15048.0       -0.25         -0.15  16.62  16.44    101.0   
42       5242.0  15650.0        1.73          2.62   7.62  10.44      4.0   
78     -16118.0  16212.0        0.24        -15.99  19.15   0.09    245.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
58     1.76        1.61    26.69     XY25      NTT  REFINERIES  
85     1.84        1.52    20.42     XY25      NTT          AC  
2     -0.01        0.64    19.99      X40      ATH      PHARMA  
42     0.33        1.45     8.33      X40      NTT        FMCG  
78    -0.99        0.60    11.22    OX40N      NTT    DURABLES

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
82  VAIBHAVGBL   521.00    53.72    59.0       H-SC     4.62  133330.0   
40        INFY  2275.00   -15.65    61.0       X-LC     7.34  328510.0   
27  HAPPSTMNDS  1488.71   -22.55    50.0       H-SC    16.62   83672.0   
15       BSOFT   836.99   -20.22    45.0       H-SC    15.45   89518.0   
30     HCLTECH  1922.01     5.38    68.0       X-LC     7.59  241121.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
82     -49445.0  160516.0        5.09        -27.05  120.39  60.77    125.0   
40      15622.0  156075.0        4.75          4.99   47.51  54.87      3.0   
27     -44493.0  154525.0        4.49        -34.72  184.68  85.85    132.0   
15     -51135.0  123911.0        3.57        -36.36  138.42  51.74    131.0   
30       -803.0   60642.0        3.35         -0.33   25.15  24.73      8.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
82    -0.31        0.94    19.42       XR      NTT  JEWELLERY  
40     0.10        2.31    11.97      X40      BTT         IT  
27    -0.29        0.59     6.81       AR      ATH         IT  
15    -0.41        0.63     3.57       XR      ATH         IT  
30    -0.01        1.70    14.85      X40      ATH         IT

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
71  SYMPHONY  1306.00   -44.18    47.0       M-SC    18.66  118830.0   
55  RAJOOENG   143.10   -41.24    47.0       H-SC    18.17   88940.0   
72     TANLA  1963.11   -28.54    34.0       H-SC    17.11  191816.0   
46  KPIGREEN   731.05     5.18    63.0       H-SC     5.29  125257.0   
65    SHALBY   327.00  1163.16    52.0       M-SC    22.63  171864.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
71     -52311.0   52261.0       -2.12        -30.57   43.98   -0.03    196.0   
55     -13560.0   54156.0       -2.09        -13.23   60.89   39.61    114.0   
72     -78945.0  408894.0       -1.67        -29.16  213.17  121.86    133.0   
46        -40.0   58971.0       -1.59         -0.03   47.08   47.03    141.0   
65      -9279.0   54739.0       -1.23         -5.12   31.85   25.10    235.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
71    -1.00        0.84     3.60    OX40N      NTT    DURABLES  
55    -0.25        0.63     8.62       AR      ATH        MISC  
72    -0.19        1.35    50.05       AR      ATH          IT  
46    -0.00        0.88    56.83       MH      ATH       POWER  
65    -0.17        1.21    37.73     XY24      NTT  HEALTHCARE

In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
61      SAIL  228.00    41.94    48.0       M-MC    10.32  224600.0   
46  KPIGREEN  731.05     5.18    63.0       H-SC     5.29  125257.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
61       -362.0  168023.0        0.37         -0.16  74.81  74.53    192.0   
46        -40.0   58971.0       -1.59         -0.03  47.08  47.03    141.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
61     -0.0        1.58    32.04     XY24      BTT    STEEL  
46     -0.0        0.88    56.83       MH      ATH    POWER

In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
79  UJJIVANSFB    60.0   118.20    66.0       H-SC    14.32  137754.0   
45   KANSAINER   340.0   -66.87    60.0       H-SC     2.18  228942.0   
78  TTKPRESTIG   770.0    96.58    50.0       M-SC     1.90   84659.0   
67         SIS   528.0  2078.76    54.0       H-SC     3.43   87924.0   
73   TATAELXSI  9161.0   -23.54    50.0       H-MC     7.63  103512.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
79      -4725.0  24245.0        1.55         -3.32  17.60  13.70    163.0   
45     -40725.0  77062.0       -0.96        -15.10  33.66  13.47    138.0   
78     -16118.0  16212.0        0.24        -15.99  19.15   0.09    245.0   
67     -23108.0  46714.0        0.07        -20.81  53.13  21.26    156.0   
73     -28323.0  70543.0        1.75        -21.48  68.15  32.03     98.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
79    -0.19        0.97    62.59    OX40N      NTT     BANKS  
45    -0.53        1.61    16.41     XY24      NTT    PAINTS  
78    -0.99        0.60    11.22    OX40N      NTT  DURABLES  
67    -0.49        0.62    18.79    OX40N      NTT      MISC  
73    -0.40        0.73    15.94    OX40N      NTT        IT

In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Empty DataFrame
Columns: [Symbol, FTT, Dev%_PE, RSI_14, Conviction, Spread%, Current, Current P/L, FTT Amt, Today P/L%, Current P/L%, FTT%, OTT%, CumlRnk, RRR Ind, CurrAlloc%, Gained%, Criteria, Strategy, Category]
Index: []

In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
56     RECLTD  446.00    47.87    56.0       M-LC     5.83  208945.0   
32   HINDZINC  730.22    29.16    52.0       M-LC    10.65  210975.0   
80  UNIONBANK  163.00   -10.08    63.0       M-LC     9.03  163603.0   
87      WIPRO  420.00   -12.89    52.0       M-LC     6.22  153276.0   
12  BANKINDIA  190.00   -20.50    72.0       H-MC    13.62  206707.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
56       5775.0   36356.0        0.73          2.84  17.40  20.74     55.0   
32       5899.0  106669.0        0.24          2.88  50.56  54.89     52.0   
80      22763.0   21726.0        1.57         16.16  13.28  31.59     66.0   
87       2331.0  107125.0        2.43          1.54  69.89  72.51     53.0   
12      26899.0   85515.0        3.18         14.96  41.37  62.52     88.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
56     0.16        1.47     8.54     XY25      NTT  FINANCE  
32     0.06        1.49    27.67      X5K      ATH   METALS  
80     1.05        1.15    46.93     XY24      NTT    BANKS  
87     0.02        1.08     7.66       XR      NTT       IT  
12     0.31        1.46    54.15       XR      NTT    BANKS

In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
12  BANKINDIA   190.00   -20.50    72.0       H-MC    13.62  206707.0   
86  WHIRLPOOL  2270.00   -38.64    65.0       M-SC     4.01  103702.0   
32   HINDZINC   730.22    29.16    52.0       M-LC    10.65  210975.0   
87      WIPRO   420.00   -12.89    52.0       M-LC     6.22  153276.0   
37  INDIAMART  4810.62   -55.39    42.0       H-SC     7.36  124788.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
12      26899.0   85515.0        3.18         14.96   41.37   62.52     88.0   
86      12204.0   66546.0       -1.06         13.34   64.17   86.07    223.0   
32       5899.0  106669.0        0.24          2.88   50.56   54.89     52.0   
87       2331.0  107125.0        2.43          1.54   69.89   72.51     53.0   
37       1452.0  130179.0       -0.93          1.18  104.32  106.72    119.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
12     0.31        1.46    54.15       XR      NTT     BANKS  
86     0.18        0.73    50.59       XR      NTT  DURABLES  
32     0.06        1.49    27.67      X5K      ATH    METALS  
87     0.02        1.08     7.66       XR      NTT        IT  
37     0.01        0.88    24.83       AR      ATH      MISC

In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
74  TATAMOTORS  1065.00   -53.17    17.0       X-LC    13.40  147614.0   
53       QUESS   986.00   -47.66    36.0       X-SC    38.76   51794.0   
24       DMART  5391.45   -15.93    39.0       X-LC    10.18  102876.0   
29     HAVELLS  2069.16    -6.60    42.0       X-MC     2.85  231457.0   
9          AWL   485.00   -65.14    45.0       X-MC     1.44  248300.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
74    -128909.0  238987.0        1.19        -46.62  161.90   39.81     54.0   
53     -13212.0  164140.0       -0.94        -20.32  316.91  232.18    198.0   
24        290.0   26521.0        0.28          0.28   25.78   26.13     38.0   
29     -16274.0   91333.0       -0.03         -6.57   39.46   30.30     92.0   
9      -53451.0  214879.0       -0.54        -17.71   86.54   53.50    116.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
74    -0.54        1.04     4.04     XY24      NTT        AUTO  
53    -0.08        0.36     1.03     XY24      NTT        MISC  
24     0.01        0.72    25.93     X40N      ATH        FMCG  
29    -0.18        1.63     5.85      X40      ATH  ELECTRICAL  
9     -0.25        1.75    10.29     XY24      NTT        FMCG

In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
66   SIEMENS  4671.50    -1.97    50.0       H-LC     0.68  157650.0   
42       ITC   452.00   -36.23    67.0       X-LC     2.09  205380.0   
85    VOLTAS  1530.00     2.98    61.0       X-MC     2.43  216195.0   
29   HAVELLS  2069.16    -6.60    42.0       X-MC     2.85  231457.0   
58  RELIANCE  1533.00    -8.78    73.0       X-LC     3.06  228727.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
66     -28445.0  75924.0        1.61        -15.29  48.16  25.51     15.0   
42       5242.0  15650.0        1.73          2.62   7.62  10.44      4.0   
85      24453.0  13296.0       -1.00         12.75   6.15  19.69     99.0   
29     -16274.0  91333.0       -0.03         -6.57  39.46  30.30     92.0   
58      18361.0  10430.0        0.07          8.73   4.56  13.68     37.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
66    -0.37        1.11    16.18       AR      ATH  ELECTRICAL  
42     0.33        1.45     8.33      X40      NTT        FMCG  
85     1.84        1.52    20.42     XY25      NTT          AC  
29    -0.18        1.63     5.85      X40      ATH  ELECTRICAL  
58     1.76        1.61    26.69     XY25      NTT  REFINERIES

In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
53       QUESS   986.00   -47.66    36.0       X-SC    38.76   51794.0   
3          ACC  3906.00   -38.07    47.0       X-MC     3.04  183900.0   
74  TATAMOTORS  1065.00   -53.17    17.0       X-LC    13.40  147614.0   
10    BAJAJHFL   181.50   -14.06    47.0       X-MC     5.68  180892.0   
29     HAVELLS  2069.16    -6.60    42.0       X-MC     2.85  231457.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
53     -13212.0  164140.0       -0.94        -20.32  316.91  232.18    198.0   
3      -54051.0  206704.0       -0.54        -22.72  112.40   64.15    174.0   
74    -128909.0  238987.0        1.19        -46.62  161.90   39.81     54.0   
10     -19614.0  116766.0       -0.36         -9.78   64.55   48.45     90.0   
29     -16274.0   91333.0       -0.03         -6.57   39.46   30.30     92.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
53    -0.08        0.36     1.03     XY24      NTT        MISC  
3     -0.26        1.30     3.16     XY24      BTT      CEMENT  
74    -0.54        1.04     4.04     XY24      NTT        AUTO  
10    -0.17        1.27     4.31     X40N      ATH     FINANCE  
29    -0.18        1.63     5.85      X40      ATH  ELECTRICAL

In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%  Current  \
53       QUESS    986.00   -47.66    36.0       X-SC    38.76  51794.0   
51     PAGEIND  51769.93   -25.96    46.0       X-MC     6.67  83590.0   
57      RELAXO   1176.00   -38.91    47.0       X-SC     4.75  83256.0   
2   ABBOTINDIA  35195.00    -8.43    48.0       X-MC     3.47  90540.0   
13   BATAINDIA   2096.00   -34.87    49.0       X-SC     4.29  97450.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
53     -13212.0  164140.0       -0.94        -20.32  316.91  232.18    198.0   
51         80.0   19953.0        0.50          0.10   23.87   23.98     82.0   
57     -61904.0  137830.0        0.06        -42.65  165.55   52.31    136.0   
2        -138.0   15048.0       -0.25         -0.15   16.62   16.44    101.0   
13     -31220.0   76518.0        0.74        -24.26   78.52   35.20    219.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
53    -0.08        0.36     1.03     XY24      NTT      MISC  
51     0.00        0.59     5.92      X40      ATH  APPARELS  
57    -0.45        0.59    11.44     X40N      NTT  FOOTWEAR  
2     -0.01        0.64    19.99      X40      ATH    PHARMA  
13    -0.41        0.69    11.85      X40      NTT  FOOTWEAR

In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
75    TCS  4389.97   -27.08    60.0       X-LC    12.49  290610.0   
40   INFY  2275.00   -15.65    61.0       X-LC     7.34  328510.0   
42    ITC   452.00   -36.23    67.0       X-LC     2.09  205380.0   
84    VBL   671.64   -16.73    59.0       X-LC    10.10  299860.0   
1     ABB  7934.00   -39.48    56.0       H-LC     6.49  252864.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
75     -55346.0  122056.0        2.82        -16.00  42.00  19.28      1.0   
40      15622.0  156075.0        4.75          4.99  47.51  54.87      3.0   
42       5242.0   15650.0        1.73          2.62   7.62  10.44      4.0   
84     -15982.0  128640.0        2.30         -5.06  42.90  35.67      5.0   
1       -8755.0  127974.0        0.53         -3.35  50.61  45.57      7.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
75    -0.45        2.05     7.43      X40      ATH          IT  
40     0.10        2.31    11.97      X40      BTT          IT  
42     0.33        1.45     8.33      X40      NTT        FMCG  
84    -0.12        2.11     8.09     X40N      ATH        FMCG  
1     -0.07        1.78     7.60       AR      NTT  ELECTRICAL

In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
7   ASIANTILES   137.00  7266.67    72.0       L-SC     7.01   80823.0   
20   COFFEEDAY    80.00   -43.66    54.0       L-SC    26.71   83405.0   
49      MASFIN   398.61   -17.34    57.0       H-SC     9.56   93780.0   
86   WHIRLPOOL  2270.00   -38.64    65.0       M-SC     4.01  103702.0   
46    KPIGREEN   731.05     5.18    63.0       H-SC     5.29  125257.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
7      -12987.0  89608.0        0.78        -13.84  110.87  81.67    269.0   
20     -30144.0  70194.0        0.91        -26.55   84.16  35.27    268.0   
49      -4200.0  25799.0        1.46         -4.29   27.51  22.05    152.0   
86      12204.0  66546.0       -1.06         13.34   64.17  86.07    223.0   
46        -40.0  58971.0       -1.59         -0.03   47.08  47.03    141.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
7     -0.14        0.57    56.52       XR      NTT  CERAMICS  
20    -0.43        0.59   104.14       XR      NTT    HOTELS  
49    -0.16        0.66    36.32       XR      ATH   FINANCE  
86     0.18        0.73    50.59       XR      NTT  DURABLES  
46    -0.00        0.88    56.83       MH      ATH     POWER

In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
62  SAMMAANCAP   326.00  -190.91    75.0       M-SC    22.62  157608.0   
5     ANGELONE  3033.00    16.84    71.0       X-SC     7.01  206993.0   
68    SONACOMS   806.63   -32.99    69.0       M-SC    10.26   83388.0   
79  UJJIVANSFB    60.00   118.20    66.0       H-SC    14.32  137754.0   
59   REPCOHOME   880.00   -55.29    71.0       H-SC     2.52  256779.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
62       8388.0  135795.0        0.00          5.62   86.16  96.62    208.0   
5       15987.0   41709.0        1.06          8.37   20.15  30.21    157.0   
68     -17771.0   57771.0        3.15        -17.57   69.28  39.54    202.0   
79      -4725.0   24245.0        1.55         -3.32   17.60  13.70    163.0   
59     -29726.0  287053.0        1.99        -10.38  111.79  89.82    134.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
62     0.06        1.11    70.57     XY25      NTT  FINANCE  
5      0.38        1.46    31.00     X40N      NTT  FINANCE  
68    -0.31        0.59    17.58       AR      ATH     AUTO  
79    -0.19        0.97    62.59    OX40N      NTT    BANKS  
59    -0.10        1.81    35.86     XY24      NTT  FINANCE

In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.64
1     25       43.98
2     50       74.82

In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.23
LC    32.99
MC    22.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.27
X40      16.09
XY25     12.32
XR       11.20
X40N      9.33
AR        7.95
OX40N     7.58
X200      1.79
X5K       1.49
SR        1.09
MH        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.44
X-LC    22.10
X-MC    17.04
M-SC    12.67
M-LC     5.19
H-LC     4.68
X-SC     4.62
H-MC     3.84
M-MC     1.58
L-SC     1.50
L-LC     1.02
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                14.81         -2.45   35.88
IT                  12.75        -16.15   78.84
FINANCE             12.19         -9.30   58.84
MISC                 6.86        -18.00   83.38
BANKS                6.56         -7.16   63.54
PAINTS               5.61        -15.88   33.15
ELECTRICAL           5.28         -9.71   48.21
INSURANCE            3.92          0.59   36.48
AUTO                 2.81        -44.11  104.26
FOOTWEAR             2.40        -29.84   81.90
TRAVEL               2.20        -42.66  114.10
AC                   2.20         -2.26   14.33
CERAMICS             2.18        -29.40   84.25
DURABLES             2.17        -18.30   43.95
BREWERIES            1.66          0.92   19.66
REFINERIES           1.61          8.03    4.56
STEEL                1.58         -0.16   74.81
METALS               1.49          2.80   50.56
ENGINEERING          1.43        -19.68   74.41
CEMENT               1.30        -29.39  112.40
CHEMICALS            1.30        -42.66  159.05
JEWELLERY            1.28        -75.62  136.64
HEALTHCARE           1.21         -5.40   31.85
MINING               1.02         -3.70   26.98
ENTERTAINMENT        0.96        -41.77  126.33
POWER                0.88         -0.03   47.08
PHARMA               0.64         -0.15   16.62
APPARELS             0.59          0.10   23.87
HOTELS               0.59        -36.14   84.16
TEXTILES             0.51        -28.57   65.23

In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3276208.0     22
XR        1321389.0     14
AR        1167157.0      9
X40        761383.0     12
X40N       601961.0      8
OX40N      571460.0     10
XY25       432302.0      8
SR         266291.0      2
X5K        106669.0      1
X200        71114.0      1
MH          58971.0      1

In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        3388919.0     25
M-SC        1209155.0     15
X-MC        1115739.0     13
X-LC        1000634.0     13
X-SC         524233.0      6
H-MC         334863.0      3
H-LC         275012.0      3
M-LC         271876.0      4
L-SC         248221.0      3
M-MC         168023.0      1
L-MC          59083.0      1
L-LC          39147.0      1

In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1162185.0      6
           AR         839071.0      5
           XR         796513.0      7
M-SC       XY24       689211.0      6
X-MC       XY24       589366.0      4
X-LC       X40        436759.0      5
           XY24       302752.0      2
X-SC       X40N       283575.0      4
H-SC       SR         266291.0      2
           OX40N      265888.0      4
X-MC       X40        248106.0      6
H-LC       AR         203898.0      2
X-LC       X40N       201620.0      3
H-MC       XY24       178805.0      1
M-MC       XY24       168023.0      1
X-SC       XY24       164140.0      1
X-MC       XY25       161501.0      2
L-SC       XR         159802.0      2
M-SC       OX40N      146610.0      4
           XY25       135795.0      1
           AR         124188.0      2
X-MC       X40N       116766.0      1
M-SC       XR         113351.0      2
M-LC       XR         107125.0      1
           X5K        106669.0      1
L-SC       OX40N       88419.0      1
H-MC       XR          85515.0      1
X-SC       X40         76518.0      1
H-LC       X200        71114.0      1
H-MC       OX40N       70543.0      1
X-LC       XY25        59503.0      3
L-MC       XR          59083.0      1
H-SC       MH          58971.0      1
L-LC       XY25        39147.0      1
M-LC       XY25        36356.0      1
           XY24        21726.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
